# SharePoint Columns – Arbeits‑Notebook (fokussiert)

> Zweck: Dieses Notebook erklärt **nur** das Modul **`graphfw.domains.sharepoint.lists.columns`** (
> Datei: `columns.py`), zeigt dessen wichtigste Methode & Parameter, demonstriert **Reloading via
> `graphfw.core.reloader.reload_df`** (nach Dateiänderungen) und liefert ausführliche sowie kurze
> Beispiel‑Snippets – inkl. gezielter **Spaltenauswahl** und **Filterung auf einen Content Type**.


## 1) Modulübersicht & API (nur `columns.py`)

**Modul**: `graphfw.domains.sharepoint.lists.columns` (Datei: `columns.py`)

**Kernfunktion**

```python
list_df(gc, *, site_url, list_title, mode="standard|extended|item", page_size=200, top=None, timeout=60, columns=None, expand=False, item_content_type=None, log=None) -> (pd.DataFrame, dict)
```

**Aufgabe**
- Ermittelt das **Spaltenschema** einer SharePoint‑Liste über Microsoft Graph. Liefert einen DataFrame mit **deterministischer Spaltenreihenfolge** sowie ein `info`‑Dict (Diagnostics).

**Wichtige Features**
- **Modi**: `standard` (nur /columns), `extended` (Items + `$expand=fields`), `item` (pro Content Type ein Beispiel‑Item).
- **GUID‑Sicherheit**: Spalte `GUID` wird sichergestellt (ggf. synthetisiert).
- **Spaltenauswahl**: `columns=[...]` matcht `internalName` **oder** `displayName` (tolerant ggü. SharePoint‑Encodings `_xNNNN_`, Groß-/Kleinschreibung etc.).
- **Diagnostics**: `info` enthält Quellen, Parameter, Warnungen, Auflösungsreport.

**Rückgabe‑Schema**
- `df`‑Spalten (fixe Reihenfolge, sofern nicht explizit gewählt): `['internalName','displayName','type','required','readOnly','hidden','indexed','enforceUnique','details','source','itemContentTypes']`
- `info`: u. a. `{ 'mode', 'list_path', 'expand', 'params', 'sources', 'warnings', 'resolution', 'module_version' }`


## 2) Reloading des Moduls (nur `columns.py`)

**Wann verwenden?**
- Immer dann, wenn Sie **Code in `columns.py` geändert** haben und das Notebook bereits läuft. Nutzen Sie dafür das **neue Hilfsmodul** `graphfw.core.reloader` und dessen Funktion **`reload_df([...])`**.

**Was passiert?**
- Es wird ein DataFrame mit den Spalten **Module | Version before | Version after** ausgegeben. Optional auch Dateipfade.


In [25]:
# Setup & Reload‑Utility (nur columns)
import sys
sys.path.insert(0, "..")
from graphfw.core.reloader import reload_df

reload_overview_df, reload_info = reload_df(
    [
        "graphfw.domains.sharepoint.lists.columns"
    ],
    show_paths=True,
    include_error=True,
)
display(reload_overview_df)
print(reload_info["errors"])


,Module,Version before,Version after,File before,File after,Error
0,graphfw.domains.sharepoint.lists.columns,2.2.0,2.2.1,c:\Users\erhard.rainer\Documents\GitHub\GRAPH_...,c:\Users\erhard.rainer\Documents\GitHub\GRAPH_...,


{}


## 3) Beispiele & Erklärungen (ausführlich)

### 1) Ziel & Kontext

Dieses Script bereitet den **zugesicherten Zugriff auf Microsoft Graph** vor und setzt die **Arbeitsziele** (SharePoint-Site und Liste), mit denen anschließend Framework-Funktionen wie `columns.list_df(...)` arbeiten können. Es baut dafür zwei zentrale Bausteine auf:

1. `TokenProvider` – kümmert sich um das Holen/Aktualisieren eines Access Tokens.
2. `GraphClient` – führt robuste HTTP-Requests gegen die Graph API aus (inkl. Retry/Backoff/Paging).

Am Ende stehen `gc` (GraphClient), `site_url` und `list_title` bereit, um weitere Domänenfunktionen aufzurufen.

---

### 2) Schritt für Schritt

#### a) Imports

```python
from graphfw.core.auth import TokenProvider
from graphfw.core.http import GraphClient
```

* **TokenProvider** lädt Anmeldedaten/Scopes und holt Tokens, typischerweise via MSAL.
* **GraphClient** kapselt HTTP-Zugriffe auf `https://graph.microsoft.com/v1.0` inkl.:

  * automatischem **Retry** bei 429/5xx,
  * **Retry-After** und **exponential Backoff**,
  * **Paging** via `@odata.nextLink`.

#### b) Konfiguration lesen

```python
CONFIG_PATH = r"C:\\python\\Scripts\\config.json"
tp = TokenProvider.from_json(CONFIG_PATH)
```

* Lädt Konfigurationswerte (Tenant, ClientId, Secret/Cert, Scopes) aus einer **lokalen JSON**.
* Vorteil: keine Secrets im Code; klare Trennung von Konfiguration und Logik.

**Beispielhafte `config.json` (vereinfachtes Schema):**

```json
{
  "tenant_id": "00000000-0000-0000-0000-000000000000",
  "client_id": "11111111-1111-1111-1111-111111111111",
  "client_secret": "***",
  "authority": "https://login.microsoftonline.com/{tenant_id}",
  "scopes": ["https://graph.microsoft.com/.default"]
}
```

> Für **Application Permissions** (App-Only) ist `/.default` üblich. Stelle sicher, dass deiner App die nötigen Graph-Rollen erteilt wurden (z. B. `Sites.Read.All` für SharePoint-Metadaten).

#### c) GraphClient erstellen

```python
gc = GraphClient(tp)
```

* Bindet den `TokenProvider` ein: jeder Request erhält ein gültiges Bearer-Token.
* Zentraler Einstiegspunkt für alle `graphfw`-Domänenmodule (SharePoint, AAD, Teams, …).

#### d) Arbeitsziel setzen (SharePoint-Kontext)

```python
site_url = "https://contoso.sharepoint.com/sites/TeamA"
list_title = "Aufgaben"
```

* **`site_url`**: vollständige Tenant-URL zur Site Collection bzw. Site.
* **`list_title`**: der **Anzeigename** der Ziel-Liste (nicht die List-ID).

Diese beiden Werte nutzt du in nachfolgenden Aufrufen, z. B. für `columns.list_df(...)`.

---

### 3) Was passiert als Nächstes?

Mit `gc`, `site_url` und `list_title` kannst du z. B.:

* **Spaltenschema laden** (Default `mode="standard"`).
* **Erweiterte Erkennung** inkl. Item-Felduniversum und optionaler Content-Type-Filter (`mode="item"`).

---

### 4) Häufige Stolpersteine & Tipps

* **Berechtigungen**: App muss die nötigen **Graph-Rollen** besitzen (Admin consent), z. B. `Sites.Read.All`.
* **Falscher Listentitel**: Wenn `list_title` nicht exakt dem Anzeigenamen entspricht, kann die Liste nicht aufgelöst werden → `info["succeeded"] == "false"`.
* **Netzwerk/Proxy**: Stelle sicher, dass der Host die Graph-Endpunkte erreicht.
* **Keine Secrets loggen**: Achte darauf, `config.json` nicht zu drucken. Unser Framework maskiert sensible Keys, trotzdem: Vorsicht bei eigenen Logs.
* **Standardmodus**: Wenn `mode` weggelassen wird, ist **`"standard"`** aktiv (lädt `/columns`). Für Content-Type-Filter **muss** `mode="item"` gesetzt werden.



In [34]:
# Auth & Client – Platzhalter (mit echten Werten ersetzen)
from graphfw.core.auth import TokenProvider
from graphfw.core.http import GraphClient

CONFIG_PATH = r"C:\\python\\Scripts\\config.json"

tp = TokenProvider.from_json(CONFIG_PATH)
gc = GraphClient(tp)
# site_url = "https://contoso.sharepoint.com/sites/TeamA"
# list_title = "Aufgaben"
site_url = r"https://michaelpachleitnergroup.sharepoint.com/sites/DataAnalytics/DataMaintenance"
list_title = r"FIBU_IC_CompanyAbbreviation"


### 3.1 Spaltentschema (Standardmodus)

#### Hinweis zum Standardmodus in `columns.py`

Die Funktion `list_df` in `graphfw.domains.sharepoint.lists.columns` hat folgenden Signaturauszug:

```python
def list_df(
    gc: GraphClient,
    *,
    site_url: str,
    list_title: str,
    mode: str = "standard",  # 'standard' | 'extended' | 'item'
    ...
)
```

#### Standardwert für `mode`

* Wird beim Aufruf kein `mode` übergeben, gilt automatisch **`mode="standard"`**.
* In diesem Modus werden die Spalten direkt über den Graph-Endpunkt `/columns` geladen.
* Der Parameter `expand=True/False` ist hier relevant.
* **Nicht** wirksam in diesem Default-Modus sind:

  * Content-Type-Filterung (`item_content_type`)
  * Item-basierte Felduniversen (die nur in `mode="item"` berücksichtigt werden).

#### Empfehlung

* Verwende den Default `mode="standard"`, wenn es nur um das reine Spaltenschema geht.
* Setze `mode="extended"`, wenn auch Details aus Item-Expands berücksichtigt werden sollen.
* Setze `mode="item"`, wenn pro Content Type die Felder ermittelt werden sollen oder du gezielt per `item_content_type` filtern möchtest.


In [40]:
from graphfw.domains.sharepoint.lists import columns

df, info = columns.list_df(
    gc,
    site_url=site_url,
    list_title=list_title,
    #mode="standard",              # default: standard
    page_size=200,
    columns=None,              # alle
    expand=False,              # nur für mode='standard' relevant
    item_content_type=None,    # optional: ID oder Name
)
df


,internalName,displayName,type,required,readOnly,hidden,indexed,enforceUnique,details,source,itemContentTypes
0,ID,ID,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
1,Created,Erstellt,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
2,Modified,Geändert,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
3,_ColorTag,Farbkennzeichnung,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
4,_ComplianceFlags,Bezeichnungseinstellung,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
5,_ComplianceTag,Aufbewahrungsbezeichnung,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
6,_ComplianceTagUserId,Bezeichnung angewendet von,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
7,_ComplianceTagWrittenTime,Aufbewahrungsbezeichnung angewendet.,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
8,_IsRecord,Element ist eine Aufzeichnung,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
9,_UIVersionString,Version,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None


In [42]:
# Rückgabe der Infos beim Ermitteln der Spalte
info

{'url': None,
 'params': {},
 'mode': 'standard',
 'list_title': 'FIBU_IC_CompanyAbbreviation',
 'site_url': 'https://michaelpachleitnergroup.sharepoint.com/sites/DataAnalytics/DataMaintenance',
 'attempt': 1,
 'retries': 0,
 'warnings': [],
 'mapping_table': {},
 'resolution_report': {'steps': [{'url': 'https://graph.microsoft.com/v1.0/sites/michaelpachleitnergroup.sharepoint.com:/sites/DataAnalytics/DataMaintenance'},
   {'url': 'https://graph.microsoft.com/v1.0/sites/michaelpachleitnergroup.sharepoint.com,1ca8f36c-bd23-485f-b2f2-d531c35ea557,7db2c8a0-dfa4-49db-a083-85fc9fe07a44/lists',
    'params': {'$select': 'id,displayName,name'}}]},
 'module_version': '2.2.1',
 'succeeded': 'true'}

### 3.2 Beispiel mit robustem Error‑Handling

Dieses Beispiel zeigt, wie du das Ergebnis `(df, info)` von `columns.list_df(...)` prüfst, Warnungen ausgibst und bei Fehlern/Teil-Erfolg entsprechend reagierst.

### Hinweise

* `succeeded == "false"` kann u. a. bedeuten: **Site/Liste nicht gefunden** oder **Content Type nicht gefunden** (wenn `mode="item"` + `item_content_type` gesetzt).
* `succeeded == "partially"` bedeutet: Liste/CT vorhanden, aber **nicht alle gewünschten Spalten** (Parameter `columns=[...]`) wurden gefunden.
* Der Default‑Modus ist `mode="standard"`. Für Content‑Type‑Filter: `mode="item"` plus `item_content_type=...` setzen.


In [44]:
from graphfw.domains.sharepoint.lists import columns
from pprint import pprint

site_url = r"https://michaelpachleitnergroup.sharepoint.com/sites/DataAnalytics/DataMaintenance"
list_title = r"TTTT"  # absichtlich so gesetzt – ggf. nicht existent

# Optionales Verhalten steuern
RAISE_ON_FALSE = True      # Exception werfen, wenn succeeded == "false"
RAISE_ON_PARTIALLY = False # Exception werfen, wenn succeeded == "partially"

# Aufruf (Default mode="standard")
df, info = columns.list_df(
    gc,
    site_url=site_url,
    list_title=list_title,
    # mode="standard",        # default: standard
    page_size=200,
    columns=None,              # alle
    expand=False,              # nur für mode='standard' relevant
    item_content_type=None,    # optional: ID oder Name (nur mode='item')
)

# --- Auswertung / Error-Handling -------------------------------------------
status = (info or {}).get("succeeded", "false")
warnings = (info or {}).get("warnings", [])
resolution = (info or {}).get("resolution_report", {})

print(f"succeeded: {status}")
if warnings:
    print("warnings:")
    pprint(warnings)

# Nützliche Zusatzinfos für die Diagnose
if resolution:
    print("\nResolution report (gekürzt):")
    # zeige z. B. letzte Steps/Fehler
    steps = resolution.get("steps", [])
    if steps:
        print("- last step:", steps[-1])
    if "error" in resolution:
        print("- error:", resolution["error"])
    if "item_mode" in resolution:
        item_mode = resolution["item_mode"]
        print("- item_mode.warning:", item_mode.get("warning"))

# Exceptions gemäß Policy auslösen
if status == "false" and RAISE_ON_FALSE:
    raise RuntimeError(
        f"columns.list_df failed for list='{list_title}' at site='{site_url}'. "
        f"warnings={warnings}"
    )

if status == "partially" and RAISE_ON_PARTIALLY:
    raise RuntimeError(
        f"columns.list_df partially succeeded for list='{list_title}'. "
        f"Some requested columns missing. warnings={warnings}"
    )

# Nur anzeigen, wenn etwas da ist
if df is not None and not getattr(df, "empty", True):
    display(df.head())
else:
    print("DataFrame ist leer.")

succeeded: false
warnings:
['site or list not found']

Resolution report (gekürzt):
- last step: {'url': 'https://graph.microsoft.com/v1.0/sites/michaelpachleitnergroup.sharepoint.com,1ca8f36c-bd23-485f-b2f2-d531c35ea557,7db2c8a0-dfa4-49db-a083-85fc9fe07a44/lists', 'params': {'$select': 'id,displayName,name'}}
- error: list not found


RuntimeError: columns.list_df failed for list='TTTT' at site='https://michaelpachleitnergroup.sharepoint.com/sites/DataAnalytics/DataMaintenance'. warnings=['site or list not found']

In [45]:
# Zurücksetzen der Listen, damit die kommenden Beispiele fuktionieren
site_url = r"https://michaelpachleitnergroup.sharepoint.com/sites/DataAnalytics/DataMaintenance"
list_title = r"FIBU_IC_CompanyAbbreviation"

### 3.3 Spaltenschema (alle Felder, Modus `extended`)

Dieser Abschnitt beschreibt **genau** die Parameter, Besonderheiten und die internen Schritte von `columns.list_df(..., mode="extended")`.

---

#### Zweck von `mode="extended"`

Der *extended*-Modus ermittelt das Feld-Universum einer SharePoint‑Liste **aus tatsächlichen Listeneinträgen** (Items), indem die Item‑Metadaten via `$expand=fields` geladen und vereinigt werden. So werden auch Felder erkannt, die in `/columns` (LISTEN‑Schema) **nicht** sichtbar sind (z. B. bestimmte System- oder dynamisch auftauchende Felder).

**Abgrenzung zu anderen Modi**

* **standard**: liest nur `/columns` (Listenschema). Schnell, aber evtl. unvollständig bei Item‑only Feldern.
* **extended** *(dieser Modus)*: liest Items mit `$expand=fields` und aggregiert deren Felder. Liefert mehr Praxisnähe.
* **item**: wie *extended*, zusätzlich CT‑Bezug (Content Types) pro Feld und optionaler Filter auf **einen** CT.

---

#### Signatur (relevant für extended)

```python
df, info = columns.list_df(
    gc,
    site_url=site_url,
    list_title=list_title,
    mode="extended",
    page_size=200,
    columns=None,
    expand=False,            # nur in mode='standard' relevant
    item_content_type=None,  # in extended ignoriert
)
```

---

#### Parameter – präzise Beschreibung

| Parameter           | Typ                                     |      Default | Gilt in extended? | Beschreibung                                                                                                                                                                                                 |
| ------------------- | --------------------------------------- | -----------: | :---------------: | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| `gc`                | `GraphClient`                           |            – |         ✅         | HTTP‑Client für Graph. Muss `get_paged(...)` und `get_json(...)` unterstützen.                                                                                                                               |
| `site_url`          | `str`                                   |            – |         ✅         | Vollständige Site‑URL, z. B. `https://contoso.sharepoint.com/sites/TeamA`.                                                                                                                                   |
| `list_title`        | `str`                                   |            – |         ✅         | **Anzeigename** der Liste. Wird case‑tolerant aufgelöst.                                                                                                                                                     |
| `mode`              | `Literal["standard","extended","item"]` | `"standard"` |         ✅         | Hier **"extended"**. Führt Item‑basiertes Felder‑Union durch.                                                                                                                                                |
| `page_size`         | `int`                                   |        `200` |         ⚠️        | Derzeit **nicht wirksam** in der Implementierung (Paging der Items erfolgt intern fix; siehe Interna).                                                                                                       |
| `columns`           | `Optional[Sequence[str]]`               |       `None` |         ✅         | Explizite Auswahl; Matching gegen `internalName` **oder** `displayName` (tolerant ggü. `_xNNNN_`, Case, Diakritik). Reihenfolge wie im Input. Fehlende Namen → `info['warnings']` & `succeeded="partially"`. |
| `expand`            | `bool`                                  |      `False` |         ❌         | **Ignoriert** in extended (wirkt nur in `mode="standard"`).                                                                                                                                                  |
| `item_content_type` | `Optional[str]`                         |       `None` |         ❌         | **Ignoriert** in extended (CT‑Filter gibt es nur in `mode="item"`).                                                                                                                                          |
| `log`               | `Any`                                   |       `None` |         ✅         | Optionaler LogBuffer; in der aktuellen Implementierung nicht zwingend genutzt.                                                                                                                               |

> **Wichtig:** In der aktuellen Version wird intern eine Item‑Stichprobe bis zu **5 Items** aus max. **50 angefragten** Items gebildet, um das Felduniversum zu bestimmen. `page_size`/`top` werden dabei nicht ausgewertet.

---

#### Rückgabe

* **`df`**: `pandas.DataFrame` mit (typisch) folgenden Spalten:

  * `internalName`, `displayName`, `type`, `required`, `readOnly`, `hidden`, `indexed`, `enforceUnique`, `details`, `source`, `itemContentTypes`
  * In *extended* stammt `source` typischerweise aus `"item/fields"` (Felder wurden über Items gefunden). Bei gemischten Quellen (wenn später kombiniert wird) könnten mehrere Quellen auftauchen – in der aktuellen Implementierung von extended jedoch rein item‑basiert.
* **`info`**: `dict` mit Diagnostik:

  * `mode`: "extended"
  * `resolution_report`: Auflösungswege (Site, Liste) und ggf. Item‑Abfragen
  * `warnings`: z. B. `missing columns: [...]`
  * `module_version`: Version der Implementierung
  * **`succeeded`** ∈ `{ "true", "false", "partially" }`:

    * `"true"`: Felder ermittelt, keine explizit angeforderten Spalten fehlen.
    * `"partially"`: Mindestens eine gewünschte Spalte (`columns=[...]`) fehlt.
    * `"false"`: Site oder Liste nicht gefunden (extended kennt keinen CT‑Fehlerfall, da `item_content_type` ignoriert wird).

---

#### Interne Arbeitsweise (aktuelle Implementierung)

1. **Site & Liste auflösen**

   * `GET /sites/{host}:{path}` → `site_id`
   * `GET /sites/{site_id}/lists?$select=id,displayName,name` → list matching per `displayName` (case‑tolerant)
2. **Items laden (Stichprobe)**

   * `GET /sites/{site_id}/lists/{list_id}/items?$top=50&$expand=fields`
   * Es werden bis zu **5 Items** verarbeitet (Performance vs. Felderabdeckung).
3. **Felder vereinigen**

   * Für jedes Item: `fields`‑Objekt keys → `internalName`/`displayName`
   * Quelle (`source`) wird als `"item/fields"` markiert; Schema‑Flags (z. B. `required`) sind item‑seitig nicht vollständig → defaulten auf `False`/`None`.
4. **Normalisierung & Heuristiken**

   * Namens‑Normalisierung (Decoding `_xNNNN_`, Case/Diakritik tolerant).
   * **GUID‑Sicherheit**: ggf. `_strip_guid(...)` auf Samples.
5. **Spaltenauswahl**

   * Wenn `columns=[...]`: exakte Reihenfolge wie angefordert; fehlende Namen → `warnings` + `succeeded="partially"`.
6. **Deterministische Reihenfolge**

   * Ohne `columns`: Sortierung über eine Prioritätsliste (`id`, `GUID`, `Created`, `Modified`, …) und sonst alphabetisch (`internalName`).

> **Leistungsaspekt:** Extended benötigt mindestens eine Item‑Abfrage mit `$expand=fields`. Das ist teurer als `/columns`, liefert aber in der Praxis ein realistischeres Bild des tatsächlichen Feldbestands in Items.

---

#### Beispiele (knapp)

**A) Alle Felder (extended)**

```python
df, info = columns.list_df(gc, site_url=site_url, list_title=list_title, mode="extended")
```

**B) Nur bestimmte Felder (Reihenfolge fest, teilweiser Erfolg → `succeeded="partially"`)**

```python
wanted = ["Title", "GUID", "Created", "Modified", "Status"]
df_sel, info_sel = columns.list_df(
    gc,
    site_url=site_url,
    list_title=list_title,
    mode="extended",
    columns=wanted,
)
```

---

#### Edge Cases & Verhalten

* **Leere/kleine Listen**: Wenn in der Stichprobe keine Items zurückkommen, ist das Ergebnis leer (`succeeded="false"`).
* **Falsch geschriebene `columns`‑Namen**: werden in `warnings` protokolliert; `succeeded="partially"`.
* **`item_content_type` gesetzt**: wird in extended **ignoriert**; CT‑Filter nur im Modus `item`.
* **`expand=True` gesetzt**: ohne Wirkung in extended; dieses Flag wirkt nur im Modus `standard` (dort `$expand=columns($expand=*)`).

---

#### Wann extended verwenden?

* Wenn du **real genutzte** Felder sehen willst (über Items), nicht nur die deklarierte Listenschema‑Sicht.
* Wenn in `standard` bestimmte Felder *nicht auftauchen*, du sie aber in den Items erwartest.
* Als Vorstufe zu `item`, falls du keinen Content‑Type‑Filter benötigst, aber mehr als `/columns` willst.



In [46]:
from graphfw.domains.sharepoint.lists import columns

df, info = columns.list_df(
    gc,
    site_url=site_url,
    list_title=list_title,
    mode="extended",              # default: standard
    page_size=200,
    columns=None,              # alle
    expand=False,              # nur für mode='standard' relevant
    item_content_type=None,    # optional: ID oder Name
)
display(df.head())
info


,internalName,displayName,type,required,readOnly,hidden,indexed,enforceUnique,details,source,itemContentTypes,sample
0,id,id,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,1
1,Created,Created,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,2025-09-04T09:21:05Z
2,Modified,Modified,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,2025-09-05T06:01:30Z
3,_ComplianceFlags,_ComplianceFlags,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,
4,_ComplianceTag,_ComplianceTag,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,


{'url': None,
 'params': {},
 'mode': 'extended',
 'list_title': 'FIBU_IC_CompanyAbbreviation',
 'site_url': 'https://michaelpachleitnergroup.sharepoint.com/sites/DataAnalytics/DataMaintenance',
 'attempt': 1,
 'retries': 0,
 'warnings': [],
 'mapping_table': {},
 'resolution_report': {'steps': [{'url': 'https://graph.microsoft.com/v1.0/sites/michaelpachleitnergroup.sharepoint.com:/sites/DataAnalytics/DataMaintenance'},
   {'url': 'https://graph.microsoft.com/v1.0/sites/michaelpachleitnergroup.sharepoint.com,1ca8f36c-bd23-485f-b2f2-d531c35ea557,7db2c8a0-dfa4-49db-a083-85fc9fe07a44/lists',
    'params': {'$select': 'id,displayName,name'}}],
  'item_mode': {'mode': 'item',
   'queries': [{'url': 'https://graph.microsoft.com/v1.0/sites/michaelpachleitnergroup.sharepoint.com,1ca8f36c-bd23-485f-b2f2-d531c35ea557,7db2c8a0-dfa4-49db-a083-85fc9fe07a44/lists/310bb462-58ea-496f-99fb-e1737194e623/contentTypes'},
    {'url': 'https://graph.microsoft.com/v1.0/sites/michaelpachleitnergroup.sharepoin

### 3.3 Spaltenschema (alle Felder, Modus `item`)

Dieser Abschnitt beschreibt **genau** die Parameter, Besonderheiten und die internen Schritte von `columns.list_df(..., mode="item")`.

---

#### Zweck von `mode="item"`

Der *item*-Modus ermittelt das Feld‑Universum einer SharePoint‑Liste **aus tatsächlichen Listeneinträgen** (Items) – wie *extended* – und ergänzt zusätzlich **Content‑Type‑Bezug**:

* Optionaler Filter auf **einen** Content Type (`item_content_type` = Name **oder** ID, z. B. `0x0101…`).
* Pro Feld kann `itemContentTypes` (Liste der CT‑IDs, in denen das Feld in den betrachteten Items vorkam) gesetzt sein.

**Abgrenzung zu anderen Modi**

* **standard**: liest nur `/columns` (Listenschema), schnell – aber ohne Item-/CT‑Sicht.
* **extended**: Item‑basierte Felder ohne CT‑Filter/-Bezug.
* **item** *(dieser Modus)*: Item‑basiert **mit** CT‑Bezug und optionalem CT‑Filter.

---

#### Signatur (relevant für item)

```python
df, info = columns.list_df(
    gc,
    site_url=site_url,
    list_title=list_title,
    mode="item",
    page_size=200,
    columns=None,
    expand=False,            # nur in mode='standard' relevant
    item_content_type=None,  # Name oder ID eines Content Types
)
```

---

#### Parameter – präzise Beschreibung

| Parameter           | Typ                                     |      Default | Gilt in item? | Beschreibung                                                                                                                                                                                     |
| ------------------- | --------------------------------------- | -----------: | :-----------: | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| `gc`                | `GraphClient`                           |            – |       ✅       | HTTP‑Client für Graph. Muss `get_paged(...)`/`get_json(...)` unterstützen.                                                                                                                       |
| `site_url`          | `str`                                   |            – |       ✅       | Vollständige Site‑URL, z. B. `https://contoso.sharepoint.com/sites/TeamA`.                                                                                                                       |
| `list_title`        | `str`                                   |            – |       ✅       | **Anzeigename** der Liste. Case‑tolerant aufgelöst.                                                                                                                                              |
| `mode`              | `Literal["standard","extended","item"]` | `"standard"` |       ✅       | Hier **"item"**. Item‑basierte Felder **mit** Content‑Type‑Bezug.                                                                                                                                |
| `page_size`         | `int`                                   |        `200` |       ⚠️      | Aktuell ohne direkte Wirkung; Item‑Sampling ist fest (siehe Interna).                                                                                                                            |
| `columns`           | `Optional[Sequence[str]]`               |       `None` |       ✅       | Explizite Auswahl; Matching gegen `internalName` **oder** `displayName` (tolerant gegenüber `_xNNNN_`, Case, Diakritik). Reihenfolge = Eingabe. Fehlende → `warnings` & `succeeded="partially"`. |
| `expand`            | `bool`                                  |      `False` |       ❌       | **Ignoriert** in item (wirkt nur in `mode="standard"`).                                                                                                                                          |
| `item_content_type` | `Optional[str]`                         |       `None` |       ✅       | **CT‑Filter**: Name **oder** ID. Wenn **nicht gefunden**, → **leerer DataFrame** und `succeeded="false"`.                                                                                        |
| `log`               | `Any`                                   |       `None` |       ✅       | Optionaler LogBuffer; nicht zwingend genutzt.                                                                                                                                                    |

> **Hinweis:** In der aktuellen Version werden für das Feld‑Universum bis zu **50 Items** angefragt, davon maximal **5 Items** tatsächlich verarbeitet (Stichprobe) – wahlweise gefiltert auf einen CT.

---

#### Rückgabe

* **`df`**: `pandas.DataFrame` mit (typisch) folgenden Spalten:

  * `internalName`, `displayName`, `type`, `required`, `readOnly`, `hidden`, `indexed`, `enforceUnique`, `details`, `source`, `itemContentTypes`
  * In *item* stammt `source` primär aus `"item/fields"` (Felder aus Items). `itemContentTypes` enthält CT‑**IDs** der Items, in denen das Feld vorkam (falls vorhanden/ermittelbar).
* **`info`**: `dict` mit Diagnostik:

  * `mode`: "item"
  * `resolution_report`: Auflösungswege (Site, Liste) und Item-/CT‑Abfragen
  * `warnings`: z. B. `content type not found: ...` oder `missing columns: [...]`
  * `module_version`: Version der Implementierung
  * **`succeeded`** ∈ `{ "true", "false", "partially" }`:

    * `"true"`: Felder ermittelt (DF nicht leer) und keine explizit angeordneten Spalten fehlen.
    * `"partially"`: Mindestens eine gewünschte Spalte (`columns=[...]`) fehlt.
    * `"false"`: Site/Liste nicht gefunden **oder** angeforderter Content Type **nicht gefunden**.

---

#### Interne Arbeitsweise (aktuelle Implementierung)

1. **Site & Liste auflösen**

   * `GET /sites/{host}:{path}` → `site_id`
   * `GET /sites/{site_id}/lists?$select=id,displayName,name` → List‑Match per `displayName` (case‑tolerant)
2. **Content Types lesen**

   * `GET /sites/{site_id}/lists/{list_id}/contentTypes`
   * **CT‑Filter aktiv?**

     * Name/ID wird **case‑/encoding‑tolerant** gesucht.
     * **Nicht gefunden** → **Abbruch** mit leerem DF, `warnings += ["content type not found: ..."]`, `succeeded="false"`.
3. **Items laden**

   * `GET /sites/{site_id}/lists/{list_id}/items?$top=50&$expand=fields`
   * Bei CT‑Filter zusätzlich: `$filter=contentType/id eq '{ct_id}'`.
   * Es werden bis zu **5 Items** verarbeitet (Sampling vs. Abdeckung).
4. **Felder vereinigen**

   * Für jedes Item: `fields`‑Objekt keys → `internalName`/`displayName`
   * Quelle (`source`) = `"item/fields"`; `itemContentTypes` enthält i. d. R. CT‑ID des jeweiligen Items.
5. **Normalisierung & Heuristiken**

   * Namens‑Normalisierung (Decoding `_xNNNN_`, Case/Diakritik tolerant).
   * **GUID‑Sicherheit**: ggf. `_strip_guid(...)` auf Samples.
6. **Spaltenauswahl**

   * Wenn `columns=[...]`: exakte Reihenfolge wie angefordert; fehlende Namen → `warnings` + `succeeded="partially"`.
7. **Deterministische Reihenfolge**

   * Ohne `columns`: Priorität (`id`, `GUID`, `Created`, `Modified`, …) → danach alphabetisch (`internalName`).

---

#### Beispiele (knapp)

**A) Alle Felder (item, ohne CT‑Filter)**

```python
df, info = columns.list_df(gc, site_url=site_url, list_title=list_title, mode="item")
```

**B) Nur bestimmter Content Type (per Name oder ID)**

```python
df_ct, info_ct = columns.list_df(
    gc,
    site_url=site_url,
    list_title=list_title,
    mode="item",
    item_content_type="Dokument"  # oder z. B. "0x0101..."
)
```

**C) Fehlerfall sichtbar machen (CT nicht gefunden)**

```python
from pprint import pprint

df_fail, info_fail = columns.list_df(
    gc,
    site_url=site_url,
    list_title=list_title,
    mode="item",
    item_content_type="NichtVorhanden",
)

print("succeeded:", info_fail.get("succeeded"))      # -> "false"
print("warnings:"); pprint(info_fail.get("warnings")) # enthält Hinweis 'content type not found'
```

---

#### Edge Cases & Verhalten

* **CT nicht gefunden**: leerer DF, `succeeded="false"`, Warnung enthalten.
* **Leere/kleine Listen (nach CT‑Filter)**: Keine Items → leeres Ergebnis, Warnung `"no items for requested scope"` (falls gefiltert), `succeeded="false"`.
* **Fehlende `columns`‑Namen**: `warnings` mit `missing columns: [...]`, `succeeded="partially"`.
* **`expand=True` gesetzt**: ohne Wirkung in item; nur in `standard` relevant.

---

#### Wann item verwenden?

* Wenn du **CT‑spezifische** Sicht benötigst (z. B. Dokument‑ vs. benutzerdefinierter CT).
* Wenn du nur Felder eines **konkreten Content Types** analysieren willst (`item_content_type=`).
* Wenn du über Items hinaus zusätzlich wissen möchtest, **in welchen CTs** Felder tatsächlich vorkommen (via `itemContentTypes`).


In [48]:
from graphfw.domains.sharepoint.lists import columns

df, info = columns.list_df(
    gc,
    site_url=site_url,
    list_title=list_title,
    mode="item",              # 'standard' | 'extended' | 'item'
    page_size=200,
    columns=None,              # alle
    expand=False,              # nur für mode='standard' relevant
    item_content_type=None,    # optional: ID oder Name
)
display(df.head())
info


,internalName,displayName,type,required,readOnly,hidden,indexed,enforceUnique,details,source,itemContentTypes,sample
0,id,id,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,1
1,Created,Created,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,2025-09-04T09:21:05Z
2,Modified,Modified,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,2025-09-05T06:01:30Z
3,_ComplianceFlags,_ComplianceFlags,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,
4,_ComplianceTag,_ComplianceTag,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,


{'url': None,
 'params': {},
 'mode': 'item',
 'list_title': 'FIBU_IC_CompanyAbbreviation',
 'site_url': 'https://michaelpachleitnergroup.sharepoint.com/sites/DataAnalytics/DataMaintenance',
 'attempt': 1,
 'retries': 0,
 'warnings': [],
 'mapping_table': {},
 'resolution_report': {'steps': [{'url': 'https://graph.microsoft.com/v1.0/sites/michaelpachleitnergroup.sharepoint.com:/sites/DataAnalytics/DataMaintenance'},
   {'url': 'https://graph.microsoft.com/v1.0/sites/michaelpachleitnergroup.sharepoint.com,1ca8f36c-bd23-485f-b2f2-d531c35ea557,7db2c8a0-dfa4-49db-a083-85fc9fe07a44/lists',
    'params': {'$select': 'id,displayName,name'}}],
  'item_mode': {'mode': 'item',
   'queries': [{'url': 'https://graph.microsoft.com/v1.0/sites/michaelpachleitnergroup.sharepoint.com,1ca8f36c-bd23-485f-b2f2-d531c35ea557,7db2c8a0-dfa4-49db-a083-85fc9fe07a44/lists/310bb462-58ea-496f-99fb-e1737194e623/contentTypes'},
    {'url': 'https://graph.microsoft.com/v1.0/sites/michaelpachleitnergroup.sharepoint.co

In [38]:
df

,internalName,displayName,type,required,readOnly,hidden,indexed,enforceUnique,details,source,itemContentTypes
0,ID,ID,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
1,Created,Erstellt,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
2,Modified,Geändert,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
3,_ColorTag,Farbkennzeichnung,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
4,_ComplianceFlags,Bezeichnungseinstellung,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
5,_ComplianceTag,Aufbewahrungsbezeichnung,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
6,_ComplianceTagUserId,Bezeichnung angewendet von,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
7,_ComplianceTagWrittenTime,Aufbewahrungsbezeichnung angewendet.,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
8,_IsRecord,Element ist eine Aufzeichnung,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
9,_UIVersionString,Version,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None


In [28]:
df

,internalName,displayName,type,required,readOnly,hidden,indexed,enforceUnique,details,source,itemContentTypes,sample
0,id,id,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,1
1,Created,Created,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,2025-09-04T09:21:05Z
2,Modified,Modified,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,2025-09-05T06:01:30Z
3,_ComplianceFlags,_ComplianceFlags,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,
4,_ComplianceTag,_ComplianceTag,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,
5,_ComplianceTagUserId,_ComplianceTagUserId,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,
6,_ComplianceTagWrittenTime,_ComplianceTagWrittenTime,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,
7,_UIVersionString,_UIVersionString,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,4.0
8,AppAuthorLookupId,AppAuthorLookupId,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,20
9,AppEditorLookupId,AppEditorLookupId,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,20


## 4) Kurzfassung (Minimal‑Snippets)


In [49]:
# Minimal – alle Spalten
from graphfw.domains.sharepoint.lists import columns
df, info = columns.list_df(gc, site_url=site_url, list_title=list_title)
df

,internalName,displayName,type,required,readOnly,hidden,indexed,enforceUnique,details,source,itemContentTypes
0,ID,ID,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
1,Created,Erstellt,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
2,Modified,Geändert,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
3,_ColorTag,Farbkennzeichnung,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
4,_ComplianceFlags,Bezeichnungseinstellung,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
5,_ComplianceTag,Aufbewahrungsbezeichnung,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
6,_ComplianceTagUserId,Bezeichnung angewendet von,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
7,_ComplianceTagWrittenTime,Aufbewahrungsbezeichnung angewendet.,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
8,_IsRecord,Element ist eine Aufzeichnung,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None
9,_UIVersionString,Version,None,False,True,False,False,False,"{'columnGroup': 'Benutzerdefinierte Spalten', ...",columns,None


## 5) Explizit bestimmte Spalten anfordern
**Ziel**: Nur die gewünschten Spalten (in genau der gewünschten Reihenfolge) zurückgeben.


In [50]:
# Gewünschte Spalten definieren
wanted = ["Title", "GUID", "Created", "Modified", "Status"]

df_sel, info_sel = columns.list_df(
    gc,
    site_url=site_url,
    list_title=list_title,
    mode="extended",      # oder 'standard'/'item'
    columns=wanted,
 )
display(df_sel)
info_sel.get("warnings")  # z. B. nicht gefundene Spalten

,internalName,displayName,type,required,readOnly,hidden,indexed,enforceUnique,details,source,itemContentTypes,sample
0,Title,Title,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,AOTG
1,Created,Created,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,2025-09-04T09:21:05Z
2,Modified,Modified,None,False,False,False,False,False,{},item/fields,[0x010033DFA309D66FC349A2CCAD11AD33240A008A548...,2025-09-05T06:01:30Z


["missing columns: ['GUID', 'Status']"]

## 6) Explizite Spaltenauswahl mit Error-Handling

Dieses Beispiel zeigt, wie du beim Aufruf von `columns.list_df(...)` **gezielt Spalten anforderst** und anschließend **robust** mit (teilweise) fehlenden Spalten umgehst.

---

### Ziel

* **Nur bestimmte Spalten** in **vorgegebener Reihenfolge** zurückgeben (via `columns=[...]`).
* **Fehlende Spalten** erkennen und melden.
* Über **Policies** entscheiden, ob bei `false`/`partially` eine Exception geworfen werden soll.

---

### Relevanter Code (inkl. Error‑Handling)

```python
from pprint import pprint
import re
import ast

# Gewünschte Spalten definieren – "Status" ist hier absichtlich (potenziell) nicht vorhanden
wanted = ["Title", "GUID", "Created", "Modified", "Status"]

# Policy: Exceptions auslösen?
RAISE_ON_FALSE = True        # wenn Site/Liste/CT nicht gefunden etc.
RAISE_ON_PARTIALLY = True    # wenn nicht alle 'wanted' Spalten gefunden wurden

df_sel, info_sel = columns.list_df(
    gc,
    site_url=site_url,
    list_title=list_title,
    mode="extended",           # oder 'standard'/'item'
    columns=wanted,            # Reihenfolge wie hier
)

status = (info_sel or {}).get("succeeded", "false")
warnings = (info_sel or {}).get("warnings", [])

print(f"succeeded: {status}")
if warnings:
    print("warnings:")
    pprint(warnings)

# fehlende Spalten aus warnings extrahieren (best effort)
missing_cols = []
for w in warnings:
    if isinstance(w, str) and w.lower().startswith("missing columns:"):
        # Erwartetes Format: "missing columns: ['X','Y']"
        payload = w.split(":", 1)[-1].strip()
        try:
            missing_cols = ast.literal_eval(payload)
        except Exception:
            # Fallback: grobe Regex auf ['...','...']
            m = re.search(r"\[(.*?)\]", payload)
            if m:
                items = [x.strip().strip("'\"") for x in m.group(1).split(",")]
                missing_cols = [x for x in items if x]

# Alternativ/zusätzlich: direkt vergleichen (falls warnings-Parsing fehlschlägt)
if not missing_cols:
    present = set(df_sel["internalName"].tolist() if "internalName" in df_sel.columns else [])
    present |= set(df_sel["displayName"].tolist() if "displayName" in df_sel.columns else [])
    missing_cols = [c for c in wanted if c not in present]

# Fehlerpolitik anwenden
if status == "false" and RAISE_ON_FALSE:
    raise RuntimeError(
        f"columns.list_df failed for list='{list_title}' at site='{site_url}'. warnings={warnings}"
    )
if status == "partially" and missing_cols and RAISE_ON_PARTIALLY:
    raise RuntimeError(
        f"columns.list_df partially succeeded: missing columns={missing_cols}. "
        f"list='{list_title}', site='{site_url}'."
    )

# Ausgabe / Weiterverarbeitung
if df_sel is not None and not getattr(df_sel, "empty", True):
    display(df_sel)
else:
    print("DataFrame ist leer.")

# Für Logs/Monitoring: fehlende Spalten immer sichtbar machen
if missing_cols:
    print("Fehlende Spalten:", missing_cols)
```

---

### Detaillierte Erklärung

#### 1) `wanted` – feste Spaltenliste

* `columns=[...]` erzwingt die **exakte Reihenfolge** im Ergebnis.
* Matching ist **tolerant**: `internalName` **oder** `displayName` (inkl. SP‑Encoding `_xNNNN_`, Case/Diakritik).

#### 2) Status & Warnings

* `info_sel["succeeded"]` ∈ `{ "true", "false", "partially" }`:

  * `false`: Site/Liste (oder CT bei `mode="item"`) nicht gefunden → Ergebnis leer.
  * `partially`: **mindestens eine** gewünschte Spalte fehlt.
  * `true`: Alle gewünschten Spalten gefunden.
* `warnings` enthält u. a. `"missing columns: [...]"`.

#### 3) Ermittlung fehlender Spalten

* Bevorzugt wird der **Warnings‑Eintrag** geparst (`ast.literal_eval` → robust und sicher bei Listen‑Strings).
* **Fallback** via Regex auf Listenmuster, falls das Parsing scheitert.
* **Notanker**: direkter Abgleich gegen `df_sel` (`internalName`/`displayName`).

#### 4) Fehlerpolitik (Policies)

* **`RAISE_ON_FALSE`**: Bei gravierendem Fehlschlag (z. B. Liste existiert nicht) → Exception.
* **`RAISE_ON_PARTIALLY`**: Wenn einzelne Spalten fehlen → Exception (bewusst streng für strikte Pipelines/Batches).

#### 5) Ausgabe/Weiterverarbeitung

* Nur anzeigen, wenn `df_sel` nicht leer ist (sonst klare Konsolenmeldung).
* Fehlende Spalten werden **immer** am Ende geloggt (gut für Monitoring/CI‑Logs).

---

### Wann diese Variante einsetzen?

* Wenn die **Vollständigkeit** einer Spaltenliste **kontraktrelevant** ist (z. B. für Exporte, Downstream‑Pipelines).
* Wenn du **explizit fehlschlagen** willst, sobald Felder fehlen (schnelles Feedback).
* Für **CI/CD‑Checks**: Das Skript kann in Jobs laufen und bei `partially`/`false` gezielt abbrechen.

---

### Anpassungen/Varianten

* **Strenger Modus**: Beide Flags auf `True` → strikt.
* **Toleranter Modus**: `RAISE_ON_PARTIALLY=False` → fehlende Spalten werden nur geloggt (kein Abbruch).
* **Modus wechseln**: `mode="standard"` (reines Listenschema) oder `mode="item"` (CT‑Bezug; setze ggf. `item_content_type`).
* **Gezielte Nachsteuerung**: Falls Spalten nur als `displayName` vorliegen, füge Alias‑Listen hinzu, z. B. `wanted_aliases = {"Status": ["State","_Status"]}` und erweitere die Matching‑Logik.


In [51]:
from pprint import pprint
import re
import ast

# Gewünschte Spalten definieren – "Status" ist hier absichtlich (potenziell) nicht vorhanden
wanted = ["Title", "GUID", "Created", "Modified", "Status"]

# Policy: Exceptions auslösen?
RAISE_ON_FALSE = True        # wenn Site/Liste/CT nicht gefunden etc.
RAISE_ON_PARTIALLY = True    # wenn nicht alle 'wanted' Spalten gefunden wurden

df_sel, info_sel = columns.list_df(
    gc,
    site_url=site_url,
    list_title=list_title,
    mode="extended",           # oder 'standard'/'item'
    columns=wanted,            # Reihenfolge wie hier
)

status = (info_sel or {}).get("succeeded", "false")
warnings = (info_sel or {}).get("warnings", [])

print(f"succeeded: {status}")
if warnings:
    print("warnings:")
    pprint(warnings)

# fehlende Spalten aus warnings extrahieren (best effort)
missing_cols = []
for w in warnings:
    if isinstance(w, str) and w.lower().startswith("missing columns:"):
        # Erwartetes Format: "missing columns: ['X','Y']"
        payload = w.split(":", 1)[-1].strip()
        try:
            missing_cols = ast.literal_eval(payload)
        except Exception:
            # Fallback: grobe Regex auf ['...','...']
            m = re.search(r"\[(.*?)\]", payload)
            if m:
                items = [x.strip().strip("'\"") for x in m.group(1).split(",")]
                missing_cols = [x for x in items if x]

# Alternativ/zusätzlich: direkt vergleichen (falls warnings-Parsing fehlschlägt)
if not missing_cols:
    present = set(df_sel["internalName"].tolist() if "internalName" in df_sel.columns else [])
    present |= set(df_sel["displayName"].tolist() if "displayName" in df_sel.columns else [])
    missing_cols = [c for c in wanted if c not in present]

# Fehlerpolitik anwenden
if status == "false" and RAISE_ON_FALSE:
    raise RuntimeError(
        f"columns.list_df failed for list='{list_title}' at site='{site_url}'. warnings={warnings}"
    )
if status == "partially" and missing_cols and RAISE_ON_PARTIALLY:
    raise RuntimeError(
        f"columns.list_df partially succeeded: missing columns={missing_cols}. "
        f"list='{list_title}', site='{site_url}'."
    )

# Ausgabe / Weiterverarbeitung
if df_sel is not None and not getattr(df_sel, "empty", True):
    display(df_sel)
else:
    print("DataFrame ist leer.")

# Für Logs/Monitoring: fehlende Spalten immer sichtbar machen
if missing_cols:
    print("Fehlende Spalten:", missing_cols)


succeeded: partially
warnings:
["missing columns: ['GUID', 'Status']"]


RuntimeError: columns.list_df partially succeeded: missing columns=['GUID', 'Status']. list='FIBU_IC_CompanyAbbreviation', site='https://michaelpachleitnergroup.sharepoint.com/sites/DataAnalytics/DataMaintenance'.

## 7) Filterung auf einen bestimmten Content Type
**Ziel**: Spaltenerkennung und Beispiel‑Item nur für **einen** Content Type durchführen.

Dieses Beispiel demonstriert, wie `columns.list_df(..., mode="item", item_content_type=...)` reagiert, wenn der **angegebene Content Type nicht existiert**, und wie du das **explizit** als Fehler behandelst.

---

### Ziel

* Einen **bewusst ungültigen** Content Type angeben (per **Name** oder **ID**) und das **Fehlerverhalten** prüfen.
* `info['succeeded']` und `info['warnings']` auswerten.
* Bei `succeeded == "false"` **gezielt** eine Exception werfen, um Pipelines/CI frühzeitig zu stoppen.

---

### Relevanter Code

```python
# Content Type absichtlich NICHT vorhanden, um den Fehlerfall zu demonstrieren
from graphfw.domains.sharepoint.lists import columns
from pprint import pprint

df_ct, info_ct = columns.list_df(
    gc,
    site_url=site_url,
    list_title=list_title,
    mode="item",
    item_content_type="NichtVorhanden",  # bewusst ungültig
)

display(df_ct.head())
print("succeeded:", info_ct.get("succeeded"))
print("warnings:")
pprint(info_ct.get("warnings"))

# Fehlersichtbar machen (optional): bei fehlendem CT explizit Exception werfen
if info_ct.get("succeeded") == "false":
    raise ValueError(
        "Content Type nicht gefunden: item_content_type='NichtVorhanden' | "
        f"warnings={info_ct.get('warnings')}"
    )
```

---

### Was passiert intern?

1. **Site & Liste auflösen**

   * `GET /sites/{host}:{path}` → `site_id`
   * `GET /sites/{site_id}/lists?$select=id,displayName,name` → Liste per `displayName` matchen.
2. **Content Types der Liste laden**

   * `GET /sites/{site_id}/lists/{list_id}/contentTypes`
   * Gesuchter CT wird **case-/encoding‑tolerant** per **Name oder ID** gesucht.
3. **Nicht gefunden?**

   * `df_ct` wird **leer** zurückgegeben.
   * `info_ct['warnings']` enthält z. B. `"content type not found: NichtVorhanden"`.
   * `info_ct['succeeded'] = "false"` (harte Fehlerklassifikation).
   * **Keine** zufälligen Felder aus Items oder `/columns` als Fallback (Bugfix-Verhalten).

---

### Auswertung & Fehlerbehandlung

* `print("succeeded:", info_ct.get("succeeded"))` → liefert hier **`"false"`**.
* `warnings` prüfen (enthält den konkreten Hinweis).
* **Optionale Exception**: Das Beispiel wirft bei `false` eine **`ValueError`** mit erläuternder Nachricht. Sinnvoll in **Pipelines/CI/CD**, um fehlerhafte Konfigurationen (falscher CT‑Name/ID) sofort zu stoppen.

> Tipp: Wenn du den **Namen** nutzt (z. B. `"Dokument"`), prüfe in Mehrsprachigkeits‑Umgebungen, ob der Anzeigename lokalisiert ist. Robustheit erreichst du mit der **CT‑ID** (z. B. `0x0101...`).

---

### Edge Cases

* **Liste existiert nicht** → `succeeded = "false"`, `warnings` enthält Auflösungshinweise (Site/List).
* **CT existiert, aber keine Items** des CT in der Stichprobe → `warnings` enthält `"no items for requested scope"`; je nach Implementierung kann `succeeded` weiterhin `"false"` sein (leeres Felduniversum).
* **`columns=[...]` zusätzlich gesetzt** → falls einige Felder fehlen, wird trotz vorhandenem CT `succeeded = "partially"` gesetzt (andere Fehlerbedingungen bleiben davon unberührt).

---

### Wann diese Variante einsetzen?

* Bei **strikten Datenverträgen**: Ein bestimmter Content Type muss vorhanden sein – sonst Abbruch.
* In **Build-/Deploy-Pipelines**: Falsche CT‑Angaben sollen **sofort** sichtbar scheitern.
* In **Notebook‑Workflows**, wenn du Debug‑Ausgaben bevorzugst und das Ergebnis bewusst leer sein soll, anstatt versehentlich mit falschen Feldern weiterzuarbeiten.



In [32]:
# Content Type absichtlich NICHT vorhanden, um den Fehlerfall zu demonstrieren
from graphfw.domains.sharepoint.lists import columns
from pprint import pprint

df_ct, info_ct = columns.list_df(
    gc,
    site_url=site_url,
    list_title=list_title,
    mode="item",
    item_content_type="NichtVorhanden",  # bewusst ungültig
)

display(df_ct.head())
print("succeeded:", info_ct.get("succeeded"))
print("warnings:")
pprint(info_ct.get("warnings"))

# Fehlersichtbar machen (optional): bei fehlendem CT explizit Exception werfen
if info_ct.get("succeeded") == "false":
    raise ValueError(
        "Content Type nicht gefunden: item_content_type='NichtVorhanden' | "
        f"warnings={info_ct.get('warnings')}"
    )


,internalName,displayName,type,required,readOnly,hidden,indexed,enforceUnique,details,source,itemContentTypes


succeeded: false
warnings:
['content type not found: NichtVorhanden']


ValueError: Content Type nicht gefunden: item_content_type='NichtVorhanden' | warnings=['content type not found: NichtVorhanden']

## 8) Troubleshooting & Hinweise
- **Import‑Fehler**: Projekt‑Root dem `sys.path` hinzufügen; Paketstruktur `graphfw/...` prüfen.
- **Berechtigungen**: Für echte Graph‑Calls benötigt Ihre App passende Scopes (Application/Delegated).
- **Keine Secrets loggen**: `TokenProvider`/Konfiguration dürfen keine Secrets ausgeben.
- **Deterministische Spalten**: Bei `columns=[...]` folgt die Reihenfolge genau Ihrer Liste; fehlende Spalten in `info['warnings']`.
